# 题目
随着新能源汽车的快速发展，充电桩的覆盖密度变得越来越重要。某汽车公司计划在一座城市内建设充电桩，其建设思路如下：

将城市划分为多个区域，每个区域建设一个充电站。
每个充电站内设有多个充电桩，且充电站之间保持合理的距离。
每个充电站可以覆盖邻近范围内的多个区域。
我们使用 n 表示区域或充电站的数量，并使用数组 station[i] 表示第 i 个充电站内充电桩的数量。给定一个范围 r，第 i 个区域可以被邻近范围内（即绝对值 |i-j| 小于等于 r，其中 0 <= i, j <= n-1）的充电站所覆盖。

对于覆盖区域 i 的充电桩数目，包括 i 区域内充电站的充电桩数量以及符合上述覆盖条件的其他区域充电站的充电桩数量。

现在，这家公司计划在该城市新增 k 个充电桩。问题是如何合理分配这 k 个充电桩到各个充电站（k 个充电桩可以增加在不同的充电站）以使得所有区域中，被覆盖最少的区域的充电桩数目得到最大化。

# 解答要求

时间限制: C/C++ 1000ms,其他语言: 2000ms

内存限制: C/C++ 50MB,其他语言: 100MB

输入

第一行输入为n，表示有n个充电站/区域，取值范围[0,100000)

第二行输入为station[n]数组，表示n个充电站中充电桩的数目[0,100000]

第三行输入为r，表示充电站可覆盖的相邻区域的范围[0,n-1]

第四行输入为k，表示需要新增的充电桩的数目[0,10000000001

输出

输出被充电桩覆盖最少的区域的充电桩的数目

样例1 \n
5

1 2 4 5 0

1

2\n
输出:
5
解释: 最优方案之一是把 2 个充电桩都建在充电站1。

每个充电站的充电桩数目分别为1 4 4 50

区域 0的充电桩覆盖数目为 1 + 4 = 5。

区域1的充电桩覆盖数目为 1 + 4 + 4 9

区域 2 的充电桩覆盖数目为 4 + 4 + 5 = 13。

区域3 的充电桩覆盖数目为 5 + 4 = 9

区域 4 的充电桩覆盖数目为 5 + 0 = 5。

充电桩覆盖数目最少是 5

无法得到更优解，所以我们返回 5。

样例2

输入:\n

4

4 4 4 4

0

3\n
输出\n
4\n
无论怎么分配新增的3个充电站，总有一个区域的充电桩覆盖数目是4
思路：
二分 + 树状数组 + 贪心。
对于每一个充电站来说，我们可以考虑成一个区间[i-r,i+r]，需要满足的是所有的区间的最小值尽可能大，因此我们二分这个最小值。那么如果某个区间小于我们二分的值的时候，此时贪心地将欠缺的充电桩添加到区间的最后一个元素，因为最后一个元素对于后面所有的区间的贡献会最大。

对于每一个区间需要做的就是查询该区间的和，但是由于区间是需要修改的，因此得使用树状数组/线段树进行维护。


树状数组：logn查询和修改前缀和数组的数据结构，按二进制拆分的思想，-x为x每位取反+1, x&-x相当于x的最低位的1。用一个数组tree[i]维护a[(i&-i)+1]到a[i&-i]的前缀和。

In [ ]:
#include<iostream>
#include <vector>
using namespace std;
int n;
int a[100010];
int R,k;

class Bit{
public:
    Bit(int n){
        trees.resize(n+1, 0);
    }
    void update(int idx, int x){
        idx++;
        while(idx< trees.size()){
            trees[idx]+=x;
            idx+=idx&-idx;
        }
    }
    int query(int idx)
    {
        int ans=0;
        idx++;
        while(idx)
        {
            ans+=trees[idx];
            idx-=idx&-idx;
        }
        return ans;
    }
    int query_range(int l, int r)
    {
        return query(r) - query(l-1);
    }
private:
    vector<int> trees;
};

bool check(int x)
{
    Bit bit = Bit(n);
    for(int i=0; i<n; i++)
        bit.update(i, a[i]);
    int res = k;
    for(int i=0;i<n;i++)
    {
        int query = bit.query_range(max(0,i-R), min(n-1, i+R));
        if(query<x)
        {
            if(query+res>=x){
                bit.update(i+R, x-query);
                res-=x-query;
            }
            else {return false;}
        }
    }
    return true;
}

int main()
{
    cin>>n;
    for(int i=0; i<n;i++)
        cin>>a[i];
    cin>>R>>k;
    int l=0, r=1000000000;
    while(l<r)
    {
        int mid=(l+r+1)>>1;
        if(check(mid))l=mid;
        else r=mid-1;
    }
    cout<<r<<endl;
}